In [1]:
fixed =  open("douban_comments_fixed.txt","w",encoding="utf-8")


lines = [line for line in open("doubanbook_top250_comments.txt","r",encoding="utf-8")]

for i, line in enumerate(lines):
    if i == 0:
        fixed.write(line)
        pre_line = ""
        continue
    
    terms = line.split("\t")
    
    if terms[0] == pre_line.split("\t")[0]:
        if len(pre_line.split("\t")) == 6:
            fixed.write(pre_line + "\n")
            pre_line = line.strip()
        else:
            pre_line = ""
    else:
        if len(terms) == 6:
            pre_line = line.strip()
        else:
            pre_line += line.strip()
            
fixed.close()







In [2]:
import csv
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def load_data(filename):
    book_comments = {}
    with open(filename,'r') as f:
        reader = csv.DictReader(f, delimiter="\t")
        for item in reader:
            book = item['book']
            comment = item['body']
            comment_words = jieba.lcut(comment)
            
            if book == "": continue
            
            book_comments[book] = book_comments.get(book,[])
            book_comments[book].extend(comment_words)
    return book_comments

if __name__ == "__main__":
    stop_words = [line.strip() for line in open("stopwords.txt","r",encoding="utf-8")]
    
    book_comments = load_data("douban_comments_fixed.txt")
    print(len(book_comments))
    
    book_names = []
    book_comms = []
    for book, comments in book_comments.items():
        book_names.append(book)
        book_comms.append(comments)
        
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    tfidf_matrix = vectorizer.fit_transform([' '.join(comms) for comms in book_comms])
    
    similarity_matrix = cosine_similarity(tfidf_matrix)
    
    book_list = list(book_comments.keys())
    print(book_list)
    book_name = input("输入图书名称：")
    book_idx = book_names.index(book_name)
    
    recommend_book_index = np.argsort(-similarity_matrix[book_idx])[1:11]
    
    for idx in recommend_book_index:
        print(f"《{book_names[idx]}》\t 相似度：{similarity_matrix[book_idx][idx]:.4f}")
        

232
['天才在左 疯子在右', '1Q84 BOOK 1', '悲伤逆流成河', '恶意', 'Harry Potter and the Deathly Hallows', '长安乱', '苏菲的世界', '许三观卖血记', '1995-2005夏至未至', '盗墓笔记', '霍乱时期的爱情', '三生三世 十里桃花', '基督山伯爵', '小时代1.0折纸时代', '洛丽塔', '1Q84 BOOK 2', '第一次的亲密接触', '神雕侠侣', '一座城池', '茶花女', '当我谈跑步时我谈些什么', '明朝那些事儿（贰）', '人类简史', '一個人住第5年', '明朝那些事儿（肆）', '寻路中国', '我们台湾这些年', '1Q84 BOOK 3', '摆渡人', '明朝那些事儿（伍）', '骆驼祥子', '盗墓笔记3', '麦琪的礼物', '格林童话全集', '水仙已乘鲤鱼去', '历史深处的忧虑', '金锁记', '草样年华', '刀锋', '飞鸟集', '七夜雪', '最初的爱情 最后的仪式', '拆掉思维里的墙', '明朝那些事儿（陆）', '追风筝的人', '小王子', '围城', '解忧杂货店', '活着', '白夜行', '挪威的森林', '嫌疑人X的献身', '三体', '不能承受的生命之轻', '红楼梦', '梦里花落知多少', '达·芬奇密码', '看见', '百年孤独', '1988：我想和这个世界谈谈', '何以笙箫默', '平凡的世界（全三部）', '简爱', '哈利·波特与魔法石', '三体Ⅱ', '飘', '送你一颗子弹', '三体Ⅲ', '傲慢与偏见', '倾城之恋', '三重门', '杜拉拉升职记', '明朝那些事儿（壹）', '哈利·波特与阿兹卡班的囚徒', '目送', '情人', '哈利·波特与密室', '万历十五年', '我们仨', '幻城', '致我们终将逝去的青春', '狼图腾', '微微一笑很倾城', '莲花', '哈利·波特与火焰杯', '边城', '月亮和六便士', '向左走·向右走', '穆斯林的葬礼', '从你的全世界路过', '天龙八部', '放学后', '哈利·波特与混血王子', '一个人的好天气', '哈利·波特与凤凰社', '喜宝', '海边的卡夫卡', '文化苦旅', '窗边的小豆豆',

In [6]:
import numpy as np

# bm25算法实现，输入为评论列表集合，k,b为超参数。输出所有评论的bm25结果矩阵
# 输入：comments = [['a','b','c'],['a','b','d'],['a','b','e']]
# 其中bm25[0] = [0.0, 0.0, 0.0, 0.0, 0.0]表示第一个评论的bm25值
# 其中bm25[0][0] = 0.0表示a的bm25值为0.0
def bm25(comments, k=1.5, b=0.75):
    # 计算文档总数
    N = len(comments)
    # 初始化文档长度列表和词频字典
    doc_lengths = []
    word_doc_freq = {}
    doc_term_dict = [{} for _ in range(N)]

    for i, comment in enumerate(comments):
        # 记录文档长度
        doc_lengths.append(len(comment))
        unique_words = set()
        for word in comment:
            # 统计词频
            doc_term_dict[i][word] = doc_term_dict[i].get(word, 0) + 1
            unique_words.add(word)
        # 统计包含该词的文档数量
        for word in unique_words:
            word_doc_freq[word] = word_doc_freq.get(word, 0) + 1

    # 计算每个单词的平均文档长度
    avg_doc_len = sum(doc_lengths) / N

    # 构建词汇表
    vocabulary = list(word_doc_freq.keys())
    word_index = {word: idx for idx, word in enumerate(vocabulary)}

    # 构建文档 - 词频矩阵
    doc_term_matrix = np.zeros((N, len(vocabulary)))
    for i in range(N):
        for word, freq in doc_term_dict[i].items():
            idx = word_index.get(word)
            if idx is not None:
                doc_term_matrix[i, idx] = freq

    # 计算 idf 值
    idf_numerator = N - np.array([word_doc_freq[word] for word in vocabulary]) + 0.5
    idf_denominator = np.array([word_doc_freq[word] for word in vocabulary]) + 0.5
    idf = np.log(idf_numerator / idf_denominator)
    idf[idf_numerator <= 0] = 0  # 避免出现 nan 值

    # 计算 bm25 值
    doc_lengths = np.array(doc_lengths)
    bm25_matrix = np.zeros((N, len(vocabulary)))
    for i in range(N):
        tf = doc_term_matrix[i]
        bm25 = idf * (tf * (k + 1)) / (tf + k * (1 - b + b * doc_lengths[i] / avg_doc_len))
        bm25_matrix[i] = bm25

    # 根据原始评论顺序重新排列 bm25 值
    final_bm25_matrix = []
    for i, comment in enumerate(comments):
        bm25_comment = []
        for word in comment:
            idx = word_index.get(word)
            if idx is not None:
                bm25_comment.append(bm25_matrix[i, idx])
        final_bm25_matrix.append(bm25_comment)

    # 找到最长的子列表长度
    max_length = max(len(row) for row in final_bm25_matrix)
    # 填充所有子列表到相同的长度
    padded_matrix = [row + [0] * (max_length - len(row)) for row in final_bm25_matrix]
    # 转换为 numpy 数组
    final_bm25_matrix = np.array(padded_matrix)

    return final_bm25_matrix

similarity_matrix_bm25 = bm25(book_comments)
# similarity_matrix_bm25
book_list = list(book_comments.keys())
print(book_list)
book_name = input("输入图书名称：")
book_idx = book_names.index(book_name)

recommend_book_index = np.argsort(-similarity_matrix_bm25[book_idx])[1:11]

for idx in recommend_book_index:
    print(f"《{book_names[idx]}》\t 相似度：{similarity_matrix_bm25[book_idx][idx]:.4f}")
    

['天才在左 疯子在右', '1Q84 BOOK 1', '悲伤逆流成河', '恶意', 'Harry Potter and the Deathly Hallows', '长安乱', '苏菲的世界', '许三观卖血记', '1995-2005夏至未至', '盗墓笔记', '霍乱时期的爱情', '三生三世 十里桃花', '基督山伯爵', '小时代1.0折纸时代', '洛丽塔', '1Q84 BOOK 2', '第一次的亲密接触', '神雕侠侣', '一座城池', '茶花女', '当我谈跑步时我谈些什么', '明朝那些事儿（贰）', '人类简史', '一個人住第5年', '明朝那些事儿（肆）', '寻路中国', '我们台湾这些年', '1Q84 BOOK 3', '摆渡人', '明朝那些事儿（伍）', '骆驼祥子', '盗墓笔记3', '麦琪的礼物', '格林童话全集', '水仙已乘鲤鱼去', '历史深处的忧虑', '金锁记', '草样年华', '刀锋', '飞鸟集', '七夜雪', '最初的爱情 最后的仪式', '拆掉思维里的墙', '明朝那些事儿（陆）', '追风筝的人', '小王子', '围城', '解忧杂货店', '活着', '白夜行', '挪威的森林', '嫌疑人X的献身', '三体', '不能承受的生命之轻', '红楼梦', '梦里花落知多少', '达·芬奇密码', '看见', '百年孤独', '1988：我想和这个世界谈谈', '何以笙箫默', '平凡的世界（全三部）', '简爱', '哈利·波特与魔法石', '三体Ⅱ', '飘', '送你一颗子弹', '三体Ⅲ', '傲慢与偏见', '倾城之恋', '三重门', '杜拉拉升职记', '明朝那些事儿（壹）', '哈利·波特与阿兹卡班的囚徒', '目送', '情人', '哈利·波特与密室', '万历十五年', '我们仨', '幻城', '致我们终将逝去的青春', '狼图腾', '微微一笑很倾城', '莲花', '哈利·波特与火焰杯', '边城', '月亮和六便士', '向左走·向右走', '穆斯林的葬礼', '从你的全世界路过', '天龙八部', '放学后', '哈利·波特与混血王子', '一个人的好天气', '哈利·波特与凤凰社', '喜宝', '海边的卡夫卡', '文化苦旅', '窗边的小豆豆', '三国